# Tensorflow入门

## 关于Tensorflow

TF核心是**数据流图（Data flow graphs）**，可用于**数值计算**。其图中的节点（Node）表示数学操作；边（Edge）则表示多维数据的流动，即张量（Tensor）。

TF的特性：

- **可移植性（Portability）**：在CPU或GPU上运行，支持台式机，服务器，手机等设备
- **性能最优化**：将硬件的计算潜能全部发挥出来
- **自动求微分**：对于基于梯度的学习算法可自动求微分

## 一般使用步骤

1. **构建图**
2. **计算误差**
3. **选择一个优化器来训练网络**
4. **执行图计算**

下面展示了一个简单的例子：

In [2]:
import tensorflow as tf
import numpy as np

# 生成假数据, 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构建图（线性模型）
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 计算误差（最小化方差）
loss = tf.reduce_mean(tf.square(y - y_data))

# 选择优化器训练网络
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 执行图计算
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 [[0.5212513  0.49353862]] [-0.20850952]
20 [[0.26298526 0.32005095]] [0.14956573]
40 [[0.15745431 0.24234638]] [0.24695499]
60 [[0.12025581 0.21493419]] [0.281296]
80 [[0.10714173 0.20526628]] [0.29340494]
100 [[0.10251807 0.20185696]] [0.29767457]
120 [[0.10088784 0.20065476]] [0.29918006]
140 [[0.10031305 0.20023088]] [0.2997109]
160 [[0.10011037 0.20008142]] [0.29989806]
180 [[0.10003892 0.20002873]] [0.29996404]
200 [[0.10001372 0.20001012]] [0.29998735]


### 构建图

TF使用图 (graph) 来表示计算任务，图中的节点被称之为**op（operation）**，一个 op获得0个或多个`Tensor`, 执行计算, 产生0个或多个`Tensor`。每个`Tensor`是一个类型化的多维数组，例如, 你可以将一小组图像集表示为一个四维浮点数数组, 这四个维度分别是`[batch, height, width, channels]`。

Python库有一个**默认图 (default graph)**，如果没有显式的声明新图，那么构建的op会被放入默认图。

如下例子里面包含3个op：两个`constant()op`, 和一个`matmul()op`，其中`constant()op`也被称为`源op（source op）`。

In [3]:
# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量 op, 产生一个 2x1 矩阵.
matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入.
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)

### 执行图计算

**构造阶段完成后, 才能启动图**. 启动图的第一步是创建一个`Session`对象, 如果无任何创建参数, 会话构造器将启动默认图. 其中`sess.run(op_output)`表示我想取某个op的输出值。

如下例子执行了图：函数调用`run(product)`会触发了图中三个op(两个常量op和一个矩阵乘法op)的执行。

In [4]:
# 'product'代表了矩阵乘法op的输出, 传入它是向方法表明, 我们希望取回矩阵乘法op的输出.
with tf.Session() as sess:
  res = sess.run(product)
  print(res)

[[12.]]


在实现上, TensorFlow将图形定义转换成**分布式执行的操作**，以充分利用可用的计算资源（如 CPU 或 GPU），一般你不需要显式指定使用 CPU 还是 GPU。你可以使用类似`with tf.device("/gpu:1"):`的方式显式的定义某个设备来执行。

## 交互式使用

使用`tf.InteractiveSession()`可以建立类似IPython之类的Python交互环境。使用`Tensor.eval()`和`Operation.run()`方法代替`Session.run()`, 这样可以避免使用一个变量来持有会话。

In [5]:
import tensorflow as tf
tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])
add_ = tf.add(x, a)

init=tf.global_variables_initializer()
init.run()

print(x.eval())
print(add_.eval())

[1. 2.]
[4. 5.]


## 变量的创建，初始化

tensorflow中有两个关于variable的op:

- `tf.Variable(initial_value=None, name=None, dtype=None, trainable=True)`: 需要给初始值的变量；
- `tf.get_variable(name, shape=None, dtype=None, initializer=None, trainable=True)`：只需要给出Shape与初始化器即可，且其可以结合Variable_scope来复用变量。

如果你在 Tensorflow 中设定了变量，那么必须使用`sess.run(tf.global_variables_initializer())`来初始化所有变量。

In [31]:
a = tf.Variable(3.1, name="a", dtype=tf.float32)
with tf.variable_scope("test", reuse=tf.AUTO_REUSE) as scope:  # 与tf.get_variable结合
    b = tf.get_variable(name="b", shape=(3, 2), dtype=tf.float32, initializer=tf.initializers.random_normal)
print(a)
print(b)

<tf.Variable 'a_25:0' shape=() dtype=float32_ref>
<tf.Variable 'test/b:0' shape=(3, 2) dtype=float32_ref>


`tf.Variable`与`tf.get_variable`的区别在于：使用`tf.Variable`时，如果检测到命名冲突，系统会自己处理（在name后面加`'1','2',...`），实际上等于建立了新的变量。而`tf.get_variable`则会报错，这个时候需要配合`tf.variable_scope()`来决定需不需要重用变量。

需要注意：`tf.Variable()`每次都在创建新对象，设置`reuse=True`和它并没有什么关系。而对于第一次创建`get_variable`变量可以设置`reuse=tf.AUTO_REUSE`。

In [36]:
# 第二次使用已经建立了的变量，可以设置reuse=True来重用
with tf.variable_scope("test", reuse=True) as scope:
    c = tf.get_variable(name="b")
print(c is b)

True


## 常量的建立

使用下面的函数来建立常量：

- `tf.constant(value,dtype=None,shape=None,name='Const')`

值可以是常量值，也可以是类型为dtype的值列表。

In [49]:
tensor1 = tf.constant([1, 2, 3])
tensor2 = tf.constant(-1, shape=[2, 3])

print(tensor1)
print(tensor2)

Tensor("Const_28:0", shape=(3,), dtype=int32)
Tensor("Const_29:0", shape=(2, 3), dtype=int32)


## 占位符Placeholder

Placeholder是Tensorflow中的占位符，暂时储存变量。如果想要从外部传入data, 那就需要用到它。其有关的函数包含两个：

- `tf.placeholder(dtype, shape=None, name=None)`: 不可以直接评估其值；
- `tf.placeholder_with_default(input, shape, name=None)`：如果输入没有fed这个值就会使用默认input的值；

In [56]:
x = tf.placeholder(dtype=tf.float32, shape=(5,))
y = tf.add(x, x)

with tf.Session() as sess:
    # print(sess.run(y))  # Error: will fail because x was not fed
    rand_x = np.random.rand(5)
    print(sess.run(y, feed_dict={x: rand_x}))

[0.6705427  1.4139662  0.26788583 0.92777485 0.1183801 ]


## 激活函数

激活函数（Activation function）用以来将输入非线性化。一般包含Relu，tanh，sigmoid等，其图形可见下图：

![](./resources/AF.png)

激励函数必须是可以微分的, 因为在 backpropagation 误差反向传递的时候, 只有这些可微分的激励函数才能把误差传递回去。

对于网络层数比较少的情况，用什么激活函数都可以；但是对于深层网络则要谨慎选择：一般CNN用Relu，RNN用Relu或tanh。